# Questão 1

Classes:
    - Promotor
    - Detrator
    - Neutro
Só 100 notícias são classificadas
Os conteúdos são rankeados por alcance
    - No grupo com os 50 maiores alcances, todos são classificados manualmente
    - O restante, 50 são selecionados aleatoriamentes e classificados manualmente
    - Alcance por grupo
    
Calcular margem de erro do índice
    

In [40]:
import cmath

In [12]:
alcance_promotor_top = 1000000 * (15/50)
alcance_detrator_top = 1000000 * (8/50)

In [13]:
alcance_promotor_res = 3000000 * (17/50)
alcance_detrator_res = 3000000 * (7/50)

In [20]:
alcance_promotor_total = alcance_promotor_top + alcance_promotor_res
alcance_detrator_total = alcance_detrator_top + alcance_detrator_res

In [23]:
indice_promotor_top = alcance_promotor_total / (alcance_promotor_top + alcance_detrator_top)
indice_promotor_res = alcance_promotor_total / (alcance_promotor_res+alcance_detrator_res)

In [24]:
indice_promotor_top

2.869565217391304

In [25]:
indice_promotor_res

0.9166666666666665

In [39]:
def margem_erro(populacao, amostra, p=0.5, z = 1.96):
    '''
    p: percentual (como não tem definido, escolhe 50% que é a pior)
    z: valor referente a tabela de distribuição da normal (referente a 95% de confiança)
    '''
    return cmath.sqrt(-(z**2*p*(1-p)*(amostra-populacao))/(amostra*populacao)).real

In [50]:
#margem de erro TOP 50
margem_erro(50,50)

0.0

In [51]:
#margem de erro Restante
margem_erro(550,50)

0.13214317304279544

In [52]:
#margem de erro de amostra 100 da população de 600
margem_erro(600,100)

0.08946135105917713

# Resposta Questão 1

A geração do índice depende do alcance de cada grupo. Para verificar a margem de erro do índice, a margem de erro dos grupos deve ser levado em consideração.

Utilizando a fórmula para descobrir o tamanho da amostra (https://pt.wikihow.com/Calcular-o-Tamanho-de-uma-Amostra) os dados de cada amostra foram utilizadas. (obs: 95% de confiança foi utilizado)

Para o grupo Top 50, como é um grupo controlado onde a população e amostra podem ser consideradas iguais, a margem de erro é 0. Já o restante, a marge de erro é cerca de 13% pois utilizo uma amostra de 50 de uma população de 550. Novamente, como o índice utiliza de ambos o grupo para compor o valor, essa margem será passada para o índice.

Se ao invés de segmentar utilizasse uma amostra de 100, a margem de erro seria menor (cerca de 8.95%) o que seria melhor para o índice.

Link de calculadoras:
- https://www.solvis.com.br/calculos-de-amostragem/
- https://pt.surveymonkey.com/mp/margin-of-error-calculator/
- https://www.opinionbox.com/calculadora-margem-de-erro/

# Questão 4

Agrupar notícias por assunto

In [1]:
import pandas as pd

pd.options.display.max_rows
pd.set_option('display.max_colwidth', 100)

In [4]:
#leitura do arquivo
df = pd.read_csv(filepath_or_buffer='agrupamento_eventos_new.csv',delimiter='\t',encoding='utf-8')

In [5]:
df.head()

,ID,Data,Título,Conteúdo,Veículo
0,51938,2020-01-22,"Com inflação no foco, juro futuro termina em leve queda na B3","Com inflação no foco, juro futuro termina em leve queda na B3 \nVictor Rezende e Lucas Hirata \n...",Valor Econômico
1,52129,2020-01-29,Reunión de la Fed impulsa a las bolsas internacionales en medio de preocupaciones por el coronav...,En medio de un entorno global en que los inversionistas siguen preocupados por la evolución del ...,Diario Financiero
2,51412,2020-01-08,Líderes: Como as novas gerações se comportam no trabalho? Diretores de LinkedIn e Mercado Livre ...,O perfil de um funcionário mudou muito nos últimos anos. Os jovens que ingressam no mercado de t...,Yahoo Brasil
3,51430,2020-01-08,BTG recebe autorização do BC para abrir escritório em Lisboa,O BTG Pactual recebeu autorização do Banco Central (BC) para abrir um escritório de representaçã...,Valor Econômico
4,51334,2020-01-06,Ibovespa recua e vai abaixo de 117 mil pontos com aumento de tensão geopolítica; Natura sobe,"SÃO PAULO (Reuters) - A bolsa paulista começava a semana com o Ibovespa em queda, contaminada pe...",Reuters Brasil


In [6]:
#verificando as datas únicas
df.Data.unique()

array(['2020-01-22', '2020-01-29', '2020-01-08', '2020-01-06',
       '2020-01-28', '2020-01-02', '2020-01-23', '2020-01-07',
       '2020-01-20', '2020-01-15', '2020-01-01', '2020-01-14',
       '2020-01-17', '2020-01-27', '2020-01-13', '2020-01-16',
       '2020-01-09', '2020-01-21', '2020-01-10', '2020-01-03',
       '2020-01-18', '2020-01-26', '2020-01-19', '2020-01-12',
       '2020-01-05', '2020-01-24', '2020-01-11', '2020-01-04',
       '2020-01-25'], dtype=object)

In [7]:
#ordenando pela data
df = df.sort_values(by=['Data'])

In [8]:
#transformando data em datetime
df['Data'] = pd.to_datetime(df['Data'])

In [9]:
#import de bibliotecas
from sklearn.feature_extraction.text import CountVectorizer
from nltk import corpus
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

#stopwords em portugues + duas palavras que não estavam incluídas
stopwords_pt = corpus.stopwords.words('portuguese')
stopwords_pt.append('ainda')
stopwords_pt.append('ser')

In [10]:
#geração do vocabulário, onde seleciono apenas palavras que aparecem em menos de 80% no documento e
#em pelo menos 2 documentos diferentes. Também são retirados as stopwords e consideram apenas palavras com mais 
#de 3 letras. Todas as palavras são colocadas em caixa baixa
def vocab_generation(df):
    count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words=stopwords_pt, lowercase=True, token_pattern=r'(?u)\b[a-zA-Z_.]{3,}\b')
    doc_term_matrix = count_vect.fit_transform(df)
    return doc_term_matrix, count_vect

In [11]:
#busca pelos melhores parâmetros para o algoritmo. Quantidade de topicos e taxa de aprendizado
def best_param(df,n_topic=[3,4,5,6,7,8,9],learning_decay=[.5, .7, .9]):
    search_params = {'n_components': n_topic, 'learning_decay': learning_decay}
    lda = LatentDirichletAllocation()
    model = GridSearchCV(lda, param_grid=search_params,n_jobs=2,cv=5)
    model.fit(df)
    return model.best_params_

In [12]:
#Modelo LDA
def lda_model(df,n_components_, learning_decay_):
    LDA = LatentDirichletAllocation(n_components=n_components_,learning_decay =learning_decay_, random_state=7)
    LDA.fit(df)
    return LDA

In [13]:
#função que recebe o dataframe, com data de início e fim, e um apelido. Retorna o dataframe com a coluna de topico
def topic_classification(df, range_start, range_end, alias):
    df_ = df.loc[(df['Data'] >= range_start) & (df['Data'] <= range_end)]
    vocab_, count_vect_ = vocab_generation(df_['Conteúdo'].values.astype('U'))
    best_param_ = best_param(vocab_)
    lda_ = lda_model(vocab_,best_param_['n_components'],best_param_['learning_decay'])
    topic_values_ = lda_.transform(vocab_)
    
    dict_={}
    for i,topic in enumerate(lda_.components_):
        dict_[i]=[count_vect_.get_feature_names()[i] for i in topic.argsort()[-3:]]
    
    df_['Topic'] = topic_values_.argmax(axis=1)
    df_['Topic'] = df_['Topic'].astype(int)
    df_['Topic']= df_['Topic'].apply(lambda x: dict_.get(x,x))
    df_['Topic']= df_['Topic'].astype(str) + ' - ' + alias
    
    return df_

In [14]:
df_week1 = topic_classification(df, '2020-01-01', '2020-01-07', 'week-1')
df_week2 = topic_classification(df, '2020-01-08', '2020-01-14', 'week-2')
df_week3 = topic_classification(df, '2020-01-15', '2020-01-21', 'week-3')
df_week4 = topic_classification(df, '2020-01-22', '2020-01-31', 'week-4')

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning

In [15]:
df_final = pd.concat([df_week1, df_week2, df_week4, df_week4], ignore_index=True)

In [16]:
df_final.head()

,ID,Data,Título,Conteúdo,Veículo,Topic
0,51284,2020-01-01,Oferta de ações na bolsa brasileira pode atingir até R$ 200 bi este ano,As empresas brasileiras devem continuar se financiando na Bolsa para promover sua expansão em 20...,Correio Popular,"['alta', 'ibovespa', 'mercado'] - week-1"
1,51271,2020-01-01,JBS: acordo de acionistas entre J&F e BNDESPar perde validade,O acordo de acionistas da JBS com o Banco Nacional de Desenvolvimento Econômico e Social (BNDES)...,Correio Popular,"['bancos', 'consignado', 'banco'] - week-1"
2,51276,2020-01-01,JBS: acordo de acionistas entre J&F e BNDESPar perde validade,O acordo de acionistas da JBS com o Banco Nacional de Desenvolvimento Econômico e Social (BNDES)...,Revista Istoé,"['bancos', 'consignado', 'banco'] - week-1"
3,51277,2020-01-01,Oferta de ações na bolsa brasileira pode atingir até R$ 200 bi este ano,As empresas brasileiras devem continuar se financiando na Bolsa para promover sua expansão em 20...,UOL,"['alta', 'ibovespa', 'mercado'] - week-1"
4,51267,2020-01-01,Oferta de ações na Bolsa brasileira pode atingir até R$ 200 bi este ano,"Oferta de ações na Bolsa pode atingir R$ 200 bilhões \nA exemplo do que ocorreu em 2019, empresa...",Estadão,"['alta', 'ibovespa', 'mercado'] - week-1"


In [17]:
df_final.to_csv('agrupamento_eventos_resposta.csv',sep=';',index=False)

# Resposta Questão 4

Foi assumido que os assuntos duram apenas uma semana, para que os tópicos não ficassem enviesado. Assim, cada semana foi quebrada em um dataset e os tópicos foram analisados com uso do algoritmo não-supervisionado LDA.

A coluna 'Topic' no dataset final contém as 3 palavras mais relevante de cada tópico encontrado, bem como um alias para identificar a forma que eles apareceram.

# Questão 5

Classificação de Mensagens

In [37]:
import sklearn.utils

In [2]:
#leitura do arquivo
df = pd.read_csv(filepath_or_buffer='classificação_publicacoes.csv',delimiter='\t',encoding='utf-8')

In [3]:
df.head()

,ID,Conteúdo,Relevante
0,40829800373,"HOUSTON , Dec. 5, 2019 /PRNewswire/ -- PAS Global LLC , the leading solution provider of indust...",0
1,39361377031,Special to the Star Tribune\n\nEastern cottonwoods are shedding innumerable seeds that are blown...,0
2,39055227972,"Future Market Insights has announced the addition of the "" "" report to their offering\n\nThis pr...",0
3,40167958715,"In Colorado, Gevo announced the signing of a Joint Development Agreement (JDA) with Leaf Resourc...",0
4,39471347875,"At first glance, it seems like a normal butcher s shop. Sausages hang in the window alongside bu...",0


In [26]:
#base desbalanceada
df.groupby('Relevante').count()

,ID,Conteúdo
Relevante,,
0,3139,3139
1,126,126


In [27]:
#separando os datasets
df_0 = df[df['Relevante']==0]

In [29]:
df_1 = df[df['Relevante']==1]

In [35]:
#criando um sample com quantidade igual
frames = [df_0.sample(126, random_state=0),df_1]
df_final = pd.concat(frames)

In [38]:
#misturando o dataset
df_final = sklearn.utils.shuffle(df_final)

In [40]:
df_final.groupby('Relevante').count()

,ID,Conteúdo
Relevante,,
0,126,126
1,126,126


In [41]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import metrics

In [42]:
X, y = df_final['Conteúdo'].values, df_final['Relevante'].values

In [43]:
#limpeza de dados
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Substituting break line
    document = re.sub(r'\n+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [58]:
#criando vetor com TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=100,min_df=2, max_df=0.8, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(documents).toarray()

In [59]:
#realizando grid search
from sklearn.model_selection import GridSearchCV
bootstrap = [False]
max_depth = [30, 40, 50]
max_features = ['sqrt']
min_samples_leaf = [1, 2, 4]
min_samples_split = [5, 10, 15]
n_estimators = [800]

param_grid = {
    'bootstrap': bootstrap,
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'n_estimators': n_estimators
}

rf = RandomForestClassifier()

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 1)

In [60]:
grid_search.fit(X, y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   45.9s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  2.1min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [61]:
#com cross-validation = 5, treinando o classificador
kf = KFold(n_splits=5)
acc=[]
pre=[]
rec=[]
f1=[]
auc=[]

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    #clf = DecisionTreeClassifier(random_state=0,max_depth=md,max_leaf_nodes=ml)
    clf = RandomForestClassifier(bootstrap=False, max_depth=grid_search.best_params_['max_depth'],
                                 max_features=grid_search.best_params_['max_features']
                                 ,min_samples_leaf=grid_search.best_params_['min_samples_leaf'],
                                 min_samples_split=grid_search.best_params_['min_samples_split'],n_estimators=800)
    
    clf.fit(X_train,y_train)

    preds = clf.predict(X_test)  
    targs = y_test
    acc.append(metrics.accuracy_score(targs, preds))
    pre.append(metrics.precision_score(targs, preds))
    rec.append(metrics.recall_score(targs, preds))
    f1.append(metrics.f1_score(targs, preds))
    auc.append(metrics.roc_auc_score(targs, preds))


print("accuracy: mean {}, std {}".format(np.mean(acc), np.std(acc)))  
print("precision: mean {}, std {}".format(np.mean(pre), np.std(pre)))  
print("recall: mean {}, std {}".format(np.mean(rec), np.std(rec)))
print("f1: mean {}, std {}".format(np.mean(f1), np.std(f1)))  
print("area under curve (auc): mean {}, std {}".format(np.mean(auc), np.std(auc)))

accuracy: mean 0.6949803921568628, std 0.06807300963320836
precision: mean 0.6786153846153846, std 0.060302418332971786
recall: mean 0.7603312629399586, std 0.058972981090160755
f1: mean 0.7147081678237759, std 0.04542881623410436
area under curve (auc): mean 0.6951089895167856, std 0.07207667308240753


In [62]:
import pickle

pickle.dump(clf, open('model_question5','wb'))

# Resposta Pergunta 5

Com um dataset desbalanceado, foi necessário realizar um undersample da classe que tinha mais valor para que não trouxesse muito vies pro classificador. Foi utilizado o classificador Random Forest por ser um ensamble, otimizando a classificação. Os resultados foram interessente se levar em consideração a pouca quantidade de dados. Para melhorar o classificador, mais dados devem ser utilizados.